# *Lab 4 - Netflix Recommendation System*

In [1]:
# Extrating features from Wikipedia

In [2]:
import pandas as pd
import numpy as np

In [3]:
link = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2019'

In [4]:
df1 = pd.read_html(link)[2]
df2 = pd.read_html(link)[3]
df3 = pd.read_html(link)[4]
df4 = pd.read_html(link)[5]

In [5]:
df = pd.concat([df1,df2,df3,df4]).reset_index(drop=True)

In [6]:
import tmdbv3api
import json
import requests

tmdb = tmdbv3api.TMDb()

In [7]:
apiauth = '22396953bf97d94aad77322a4980e2d4'
tmdb.api_key = apiauth

In [8]:
from tmdbv3api import Movie

def get_genre(x):
    genres = []
    result = Movie().search(x)
    movie_id = result[0].id
    response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
    data_json = response.json()
    if data_json['genres']:
        for i in range(0, len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return ' '.join(genres)
    else:
        np.nan

In [9]:
# Extract genres

In [10]:
df['genres'] = df['Title'].apply(lambda x:get_genre(str(x)))

In [11]:
df2019 = df[['Title','Cast and crew','genres']].copy()

In [12]:
# Extract Director

In [13]:
def get_directors(x):
    if '(director)' in x:
        return x.split('(director); ')[0]
    elif '(director/screenplay)' in x:
        return x.split('(director/screenplay); ')[0]
    else:
        return x.split('(directors); ')[0]

In [14]:
df2019['directors'] = df2019['Cast and crew'].apply(lambda x:get_directors(str(x)))

In [15]:
# Extract Actor 1

In [16]:
def get_actor1(x):
    if 'screenplay)' in x:
        return x.split('screenplay); ')[-1].split(',')[0]

In [17]:
df2019['actor_1_name'] = df2019['Cast and crew'].apply(lambda x:get_actor1(str(x)))

In [18]:
# Extract Actor 2

In [27]:
def get_actor2(x):
    if len(x.split('screenplay);')[-1].split(', '))< 2:
        return np.nan
    else:
        return x.split('screenplay);')[-1].split(',')[1]

In [28]:
df2019['Cast and crew']

0      Adam Robitel (director); Bragi F. Schut, Maria...
1      Jen McGowan (director); Julie Lipson (screenpl...
2      Wilson Coneybeare (director/screenplay); Donal...
3      Charles Martin Smith (director); W. Bruce Came...
4      Neil Burger (director); Jon Hartmere (screenpl...
                             ...                        
240    Nick Bruno, Troy Quane (directors); Brad Copel...
241    Greta Gerwig (director/screenplay); Saoirse Ro...
242    Sam Mendes (director/screenplay); Krysty Wilso...
243    Destin Daniel Cretton (director/screenplay), A...
244    Chinonye Chukwu (director/screenplay); Alfre W...
Name: Cast and crew, Length: 245, dtype: object

In [29]:
df2019['actor_2_name'] = df2019['Cast and crew'].apply(lambda x:get_actor2(str(x)))

In [30]:
df2019['actor_2_name']

0               Logan Miller
1                Jay Paulson
2         Vincent Kartheiser
3         Edward James Olmos
4                 Kevin Hart
               ...          
240              Tom Holland
241              Emma Watson
242     Dean-Charles Chapman
243               Jamie Foxx
244           Wendell Pierce
Name: actor_2_name, Length: 245, dtype: object

In [ ]:
# Extract Actor 3

In [31]:
def get_actor3(x):
    if len(x.split('screenplay);')[-1].split(', '))< 3:
        return np.nan
    else:
        return x.split('screenplay)')[-1].split(',')[2]

In [32]:
df2019['actor_3_name'] = df2019['Cast and crew'].apply(lambda x:get_actor3(str(x)))

In [33]:
df2019.rename(columns={'Title':'movie_title'}, inplace=True)

In [35]:
df2019_ = df2019.drop('Cast and crew', axis=1).copy()

In [36]:
df2019_['actor_1_name'].replace(np.nan, 'unknown',inplace=True)

In [37]:
df2019_['actor_2_name'].replace(np.nan, 'unknown',inplace=True)

In [38]:
df2019_['actor_3_name'].replace(np.nan, 'unknown',inplace=True)

In [40]:
df2019_['movie_title'] = df2019_['movie_title'].str.lower()

In [43]:
df2019_.rename(columns={'directors':'director_name'},inplace=True)

In [44]:
df2019_['Combined'] = df2019_['movie_title'] + df2019_['actor_1_name']+ df2019_['actor_2_name']+ df2019_['actor_3_name']+df2019_['director_name']+ df2019_['genres']

In [47]:
df2019_.isna().sum()

movie_title      0
genres           0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
Combined         0
dtype: int64

In [45]:
df2019_.to_csv('Processed/FinalData3.csv', index=False)